## Algorithm

The algorithm used to solve the environment is Double Deep Q learning as presented [here](https://arxiv.org/pdf/1509.06461.pdf)

The [Deep Q learning](https://web.stanford.edu/class/psych209/Readings/MnihEtAlHassibis15NatureControlDeepRL.pdf) uses multi-layered neural network to approximate the state-action value function $\mathcal{Q}$. This network takes state $s$ as input and provides apporximation to $\mathcal{Q(s,a)}$ for all the discrete actions $a$.   
To stablize the training process it introduces two very important ideas:
- **Replay Memory**: All the one step interactions( or multi-step if multistep returns is used) are stored in a memory called replay buffer/memory. To update network a batch of transitions is sampled uniformly from this buffer after sufficient number of transitions are available in the replay buffer. This step randomizes over data and helps to remove the correlations present in sequentially gathered data.
- **Target Network**: A target network is same as online deep Q network, except that its parameters are copied from the online network after fixed period time $\tau$. Between periods its parameters remain fixed. Thus it reduces correlation with target value.

If $\theta$ represents local online network parameters and $\theta^{'}$ target network parameters then the loss function at iteration $i$ is:   
$\mathcal{L}_{i}(\theta_{i}) = \mathbb{E}_{(s,a,r,s{'})\sim\mathcal{U(D)}}\lbrack(r + \gamma max_{a_{'}}\mathcal{Q}(s^{'},a^{'};\theta^{'}_{i}) - \mathcal{Q}(s,a;\theta_{i}))^{2}\rbrack$   

The max operator in Deep Q Learning is used for both selecting an action and evaluatinf its value. It has been [shown](https://arxiv.org/pdf/1509.06461.pdf) that this results in overestimation of state-action values $\mathcal{Q}$. Overestimation of state-action value can result in sub-optimal performance/learning. This over-estimation can be avoided by seperating the selection of an action and evaluating corresponding state-action value. This is the idea behind Double Deep Q learning.   

In double deep Q learing the local online Q network is used for selecting an action value and the target network is used for evaluating  corresponding state-action value. This small change helps greatly in reducing the over-estimation of state-action values.   
The loss for Double deep Q learning is:   
$\mathcal{L}_{i}(\theta_{i}) = \mathbb{E}_{(s,a,r,s{'})\sim\mathcal{U(D)}}\lbrack(r + \gamma\mathcal{Q}(s^{'},a_{max};\theta^{'}_{i}) - \mathcal{Q}(s,a;\theta_{i}))^{2}\rbrack$   
where, $a_{max} = argmax_{a^{'}} \mathcal{Q}({s^{'},a^{'};\theta_{i}})$

### Algorithm Implemented

1. `Initialize a local Q-network and a target Q-network`.
  - `local Q-network parameters` $\theta$ `and target Q-network parameters` $\theta^{'}$. `Initially` $\theta^{'} = \theta$
2. `Initialize a Replay Buffer` $\mathcal{D}$.
3. `Intialize the start value of epsilon` $\epsilon = 1$
4. `for episode = 1 to M:`
  - `Reset environment and get initial state` $s$
  - `set value of step` $step = 0$
  - `Untill the episode is complete do:`
    - `An agent chooses an action as follows:`
      - $a_{max} = argmax_{a^{'}} \mathcal{Q}(s,a^{'};\theta)$   
      `with probability` $1-\epsilon, a = a_{max}$    
      `with probability`$\epsilon, a = \mathcal{U}(0,1,2,3)$
    - `Perform action` $a$ `and obtain reward` $r$ `and next state` $s^{'}$
    - `Store `$(s,a,r,s^{'})$ `in` $\mathcal{D}$
    - $s = s^{'}$
    - `update step value ` $step = (step + 1) \% $ `UPDATE-EVERY` 
    - `if len`$(\mathcal{D})$ ` > UPDATE_AFTER and` $step == 0$   
      - `Sample a mini-batch ` $\mathcal{B}$ `from` $\mathcal{D}$.   
      $\mathcal{B}= (s^{j},a^{j},r^{j},s^{'j})$    
      `where,` $s^{j}$: `states`    
      $a^{j}$: `actions`    
      $r^{j}$: `rewards`    
      $s^{'j}$: `next states`    
      - `Set` $target= r^{j} + \gamma\mathcal{Q}(s^{'j},a_{max}^{j};\theta^{'})$   
      `where, ` $a_{max}^{j} = argmax_{a^{'}} \mathcal{Q}({s^{'j},a^{'};\theta})$
      - `Update local Q-network by minimizing the following loss:` $\frac{1}{B} \sum_{j}(target - Q(s^{j},a^{j};\theta))^{2}$
      - `Update target network parameters:`
      - $\theta^{'} = \tau \theta + (1-\tau)\theta^{'}$


## Hyper-parameters

- BUFFER_SIZE = $10^{5}$
   - Size of the replay buffer
- BATCH_SIZE = 64 
   - number of instances sampled in a batch
- GAMMA = 0.99
   - Discount factor
- TAU = $10^{-3}$ 
   - Parameter for soft update of the target network parameters
- LR = $5\times10^{-4}$
   - Learning rate for critic network
- UPDATE_AFTER = 64
   - Minimum number of samples in the buffer after which learning can start
- UPDATE_EVERY = 4
   - Network updation period
- $\epsilon_{start}$ = 1.0
   - initial value of $\epsilon$
- $\epsilon_{end}$ = 0.01
   - Final value of $\epsilon$

### Network Architecture

- Q-Network takes state $s$ of an agent as its input. It outputs $\mathcal{Q}(s,a)$ for all $a \in \{0,1,2,3\}$  
- Individual State is of size 37, so a tensor of shape (33,)becomes input to the first linear layer(fc1) of size 64. It is followed by a ReLU unit.
- Second linear layer(fc2) is of size 64. It takes output of `fc1 + ReLU` as input. It is followed by a ReLU layer.
- Third linear layer(fc3) is of size 4, It takes output of `fc2 + ReLU` as input. It is final layer of the network.

Detailed arcitecture is given below

1. **fc1**: Linear(in_features=37, out_features=64, bias=True)
2. **ReLU** layer
3. **fc2**: Linear(in_features=64, out_features=64, bias=True)
4. **ReLU** layer
5. **fc3**: Linear(in_features=64, out_features=4, bias=True)

![NetworkArchitecture](./network-architecture.png)

### Plot of the Rewards

**The environment is solved in 382 episodes**

![Result](./result.png)

### Future Work

- **Tuning Hyperparameters**: Trying different setting of hyper-parameters and presenting how changing them have impact on learning time or if the model diverges for some setting.
- **Priority Replay Buffer**: Using replay memory with priority based sampling. It might speed up the training.
- **Dueling Architecture**: Using dueling architecture for solving the environment and compare the results.

### Trained agents playing
![AgentPlaying](./trained_agent.gif)

[//]: # (Image References)

[image1]: ./sample_play.gif "Trained Agent"


[//]: # (Image References)

[image1]: ./sample_play.gif "Trained Agent"